In [ ]:
import numpy as np
import sklearn
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal
from typing import Tuple

import warnings
warnings.filterwarnings('ignore')

# Матричные производные

На вход подаются вектор $\boldsymbol{x}$ длины $n$, $\boldsymbol{c}$ длины $m$ и матрица $A$ размера $\{n\times m\}$

Пусть $y = \boldsymbol{x}^TA\boldsymbol{c}$

Найдите $\frac{dy}{d\boldsymbol{x}}$, $\frac{dy}{dA}$ и $\frac{dy}{d\boldsymbol{c}}$

In [ ]:
def matrix_deriv(x: np.array, A: np.array, c: np.array):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [ ]:
######################################################
x = np.array([ [1], [2], [3],  [4]])
c = np.array([[25], [5],[-9]])
A = np.arange(12).reshape(4,3)

y1, y2, y3 = matrix_deriv(x, A, c)

assert_array_equal(y1, np.array([[-13],
                                [ 50],
                                [113],
                                [176]]))

assert_array_equal(y2, np.array([[ 25,   5,  -9],
                                [ 50,  10, -18],
                                [ 75,  15, -27],
                                [100,  20, -36]]))

assert_array_equal(y3, np.array([[60, 70, 80]]))
######################################################


# Честная регрессия

В случае линейной регрессии по функционалу $MSE$ задачу оптимизации можно записать следующим образом:

$$ \frac{1}{n}\sum_{k=1}^N (\boldsymbol{x}_k^T\boldsymbol{w} - y_k) ^ 2 \to \min_{\boldsymbol{w}}$$

Мы уже знаем, что решение будет выглядеть следующим образом:

$$\boldsymbol{w} = \begin{cases} X^{-1}\boldsymbol{y}, & n = m\\
(X^TX)^{-1}X^T\boldsymbol{y}, & n > m\\
X^{T}(XX^{T})^{-1}\boldsymbol{y}, & n < m
\end{cases}$$

где $n$ - количество объектов, а $m$ - количество признаков + 1 (дополнительно 1 вес без признака).

Решите задачу регрессии честно, используя формулу выше. 

Чтобы вы не запутались мы сразу написали добавление единичного столбца к матрице $X$.


### Sample 1
#### Input:
```python
X_train = np.array([[1]])
y_train = np.array([2])
X_test = np.array([[0.], [2.], [3.]])
```
#### Output:
```python
model.predict(X_test) == np.array([1., 3., 4.])
model.w_ == np.array([[1.], [1.]])
model.coef_ == np.array([1., 1.])
```

In [ ]:
class TrueLinReg():
    def __init__(self):
        self.w_ = None  # столбец
        self.coef_ = None # строка

    def fit(self, X: np.array, y: np.array) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1) # добавляем вес без признака

        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        self.coef_ = self.w_.reshape(-1)
        return self

    def predict(self, X) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        return (X @ self.w_).reshape(-1)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = TrueLinReg().fit(X_reg, y_reg)

assert_array_almost_equal(model.predict(np.array([[0], [3], [4]])), np.array([0, 3, 4]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[1.], [0.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([1., 0.]), decimal=2)
######################################################
X_reg = np.array([[1], [2], [3]])
y_reg = np.array([1, -2, 1])

model = TrueLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[0],[4]])), np.array([0., 0.]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[0.], [0.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([0., 0.]), decimal=2)
######################################################
X_reg = np.array([[1]])
y_reg = np.array([2])

model = TrueLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[0.], [2.], [3.]])), np.array([1., 3., 4.]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[1.], [1.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([1., 1.]), decimal=2)
######################################################
X_reg, y_reg = make_regression(n_samples=10, n_features=15, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model_my = TrueLinReg().fit(X_reg, y_reg)
coef_real = np.hstack([model.coef_, model.intercept_])

assert sklearn.metrics.mean_absolute_error(model_my.coef_, coef_real) < 20
######################################################


# Честная регуляризация

Добавьте регуляризацию с коэффициентом $\lambda$ и решите задачу регрессии (для любого соотношения $m$ и $n$ используйте формулу)
$$\boldsymbol{w} = (X^TX + \lambda E)^{-1}X^T\boldsymbol{y}$$
где $E$ - единичная матрица.

Не забудьте сами добавить **справа** единичный столбец к матрице $X$ аналогично предыдущей задаче.
### Sample 1
#### Input:
```python
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])
lamb = 1

X_test = np.array([[0], [4]])
```
#### Output:
```python
model.predict(X_test) == np.array([0.33, 3])
model.w_ == np.array([[0.67], [0.33]])
model.coef_ == np.array([0.67, 0.33])
```

In [ ]:
class TrueReg():
    def __init__(self, lamb):
        self.lamb = lamb
        
    def fit(self, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
        
    def predict(self, X):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = TrueReg(1).fit(X_reg, y_reg)

assert_array_almost_equal(model.predict(np.array([[0], [4]])), np.array([0.33, 3]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[0.67], [0.33]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([0.67, 0.33]), decimal=2)
######################################################
X_reg, y_reg = make_regression(n_samples=100, n_features=99, n_targets=1)

model = Ridge(1).fit(X_reg, y_reg)
model_my = TrueReg(1).fit(X_reg, y_reg)
coef_real = np.hstack([model.coef_, model.intercept_])

assert_array_almost_equal(model_my.coef_, coef_real, decimal=0)
assert sklearn.metrics.mean_absolute_error(model_my.coef_, coef_real) < 1
######################################################


# Градиент

На вход подаются обучающая выборка $(X,\boldsymbol{y})$ и как-то определенные веса $w$. Верните градиент функции $MSE$ для изменения весов в алгоритме градиентного спуска.
$$\nabla_{\boldsymbol{w}} L = \frac{2}{n} X^{T}(X\boldsymbol{w} - \boldsymbol{y})$$

Здесь не нужно добавлять единицы сбоку к матрице, считаем, что они уже есть.

### Sample 1
#### Input:
```python
w = np.array([[0.2], [0.2]]) # столбец!
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[2],[3]]) # столбец!
```
#### Output:
```python
array([[-5.6],
       [-5.6]]) # возвращаем столбец!
```

In [ ]:
def gradient_step(w: np.array, X: np.array, y: np.array) -> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [ ]:
######################################################
w = np.array([[0.2], 
              [0.2]])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[2],[3]])
assert_array_almost_equal(gradient_step(w, X, y), np.array([[-5.6], [-5.6]]), decimal=3)
######################################################


# Стохастический градиент

Реализуйте стохаистический градиентный спуск: пересчитайте $\nabla_{\boldsymbol{w}}L$ только для одного случайно выбранного элемента из выборки $X$.

$$L_{k} = (\boldsymbol{x}_k^T \boldsymbol{w} - y_k) ^ 2 \to \min_{\boldsymbol{w}}$$

$$\nabla_{\boldsymbol{w}}L_{k} = \boldsymbol{x}_k (\boldsymbol{x}_k^T \boldsymbol{w}  - y_k)$$
где $\boldsymbol{x}_i$ - вектор объекта выборки, выбранный случайно.  


Здесь не нужно добавлять единицы сбоку к матрице $X$, считаем, что они уже есть.

### Sample 1
#### Input:
```python
w = np.array([[1], [1], [1]]) #столбец!
X = np.array([[1, 1, 1], [0, 0, 0]])
y = np.array([[1], [0]]) #столбец!
```
#### Output:
```python
array([[0.],
       [0.],
       [0.]]) #столбец!
или
array([[2.],
       [2.],
       [2.]]) #столбец!
```

In [ ]:
import numpy as np

def stochastic_step(w: np.array, X: np.array, y: np.array) -> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [ ]:
######################################################
w = np.array([[1], [1], [1]])
y = np.array([[1], [0]])
X = np.array([[1, 1, 1], [0, 0, 0]])

z , k = 0 , 0
for i in range(100):
    g = stochastic_step(w, X, y)
    if g[0] == 2:
        z += 1
    else:
        k += 1
assert z >= 35 and k >= 35
######################################################


# Градиентный спуск

Теперь реализуем Линейную регрессию на градиентном спуске. Реализуйте пересчёт весов в цикле для алгоритма градиентного спуска. Начальные веса инициализированы нулями. 
$$\boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \eta\nabla_{\boldsymbol{w}} L$$

Используйте параметры `max_iter`=$1000$, `eta`=$0.1$

Все итерации проходить не нужно. Остановитесь в момент, когда норма разницы между старыми и новыми весами станет меньше $1e-9$. 


### Sample 1
#### Input:
```python
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])

model = GDLinReg(max_iter=1000, eta=0.1).fit(X_train, y_train)
y_pred = model.predict(np.array([[3],[4]]))

```
#### Output:
```python
y_pred = np.array([3., 4.])
model.coef_ = np.array([1., 0.])
```

In [ ]:
class GDLinReg():
    def __init__(self, max_iter=1000, eta=0.1):
        self.max_iter = max_iter
        self.eta = eta
        self.coef_ = None #строка
        self.w_ = None #столбец
    
    def _gradient_descending(w, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def fit(self, X, y):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        n, m = X.shape
        self.w_ = np.zeros((m, 1)) #столбец
        
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)   
        return (X @ self.w_).reshape(-1) #возвращаем всегда строку

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_regression
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = GDLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[3],[4]])), np.array([3., 4.]), decimal=1)

assert_array_almost_equal(model.coef_, np.array([1., 0.]), decimal=1)
######################################################
X_reg, y_reg = make_regression(n_samples=200, n_features=10, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model2 = GDLinReg().fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])
coef_my = model2.coef_

assert mean_absolute_error(coef_my, coef_real) < 5
######################################################


# Регуляризация

Реализуйте L2-регуляризацию (так же известную как Ridge).

$$L = \frac{1}{2}\lVert X\boldsymbol{w} - \boldsymbol{y}\rVert_2^2 + \frac{\lambda}{2}\lVert \boldsymbol{w} \rVert_2^2$$

Найдите новый $\nabla_{\boldsymbol{w}} L$ и верните его.

Обратите внимание, что свободный коэффициент весов (самый последний) **не нужно** регуляризовывать.

### Sample 1
#### Input:
```python
w = np.array([[1], [1]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
```
#### Output:
```python
np.array([[28.], 
          [27.]]) #возвращаем столбец!
```

In [ ]:
def gradient_step_l2(w: np.array, X: np.array, y: np.array, lamb: float) -> np.array:
    # Коэффициент регуляризации lamb
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [ ]:
######################################################
w = np.array([[1], [1]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
assert_array_almost_equal(gradient_step_l2(w, X, y, 1), np.array([[28.], [27.]]))
######################################################


# Логистический градиент

Теперь переходим к задаче классификации и Логистической регрессии. Мы уже знаем, что в нашем случае нужно минимизировать метрику $LogLoss$:
$$L = -\ln{Likelihood} = -\sum_{k=1}^{n}\Big(y_k\ln{(\sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))} + (1 - y_k)\ln{(1 - \sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))\Big)}$$
где $\sigma(z) = \frac{1}{1 + e^{-z}}$, $y \in \{0,1\}$

Ваша задача взять **производную** этой функции и вернуть вектор градиентов $\nabla_{\boldsymbol{w}}\ln{L}$. Формулу необходимо вывести в векторном виде, чтобы считалось быстрее.

Обратите внимание: 

* $w$, $y$ - столбцы, а не строки. Ответ тоже столбец.
* В функции используется **натуральный** логарифм.
* Нужно посчитать полный градиент, а не стохастический.

### Sample 1
#### Input:
```python
w = np.array([[0], [0]])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[0],[0]])
```
#### Output:
```python
array([[2], 
       [2]]) #возвращаем столбец!
```

In [ ]:
import numpy as np

def log_gradient_step(w: np.array, X: np.array, y: np.array)-> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [ ]:
import numpy as np
from numpy.testing import assert_array_almost_equal

######################################################
w = np.array([[0], [0]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
assert_array_almost_equal(log_gradient_step(w, X, y), np.array([[2], [2]]))
######################################################
w = np.array([[1], [2]])
X = np.array([[11,21], [12,22], [13,31]])

y = np.array([[3],[2],[10]])
assert_array_almost_equal(log_gradient_step(w, X, y), np.array([[-151.],
                                                                [-343.]]))
######################################################


# Логистическая регрессия

Теперь реализуем Логистическую регрессию на стохастическом градиентном спуске.
$$L = -\ln{Likelihood} = -\sum_{k=1}^{n}\Big(y_k\ln{(\sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))} + (1 - y_k)\ln{(1 - \sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))\Big)} + \frac{\lambda}{2}\lVert \boldsymbol{w} \rVert_2^2 \rightarrow \min$$
где $\sigma(z) = \frac{1}{1 + e^{-z}}$, $y \in \{0,1\}$, $\lVert \boldsymbol{w} \rVert_2^2 = \sum_{i=1}^{m} w_i^2$= - квадрат эвклидовой нормы

Собственно вероятность принадлежности определенному классу:
$$P(y_k=1) = \sigma(\boldsymbol{x}_k^T \boldsymbol{w})~~~P(y_k=0) = 1 - \sigma(\boldsymbol{x}_k^T \boldsymbol{w})$$

Реализуйте пересчёт весов в цикле для алгоритма градиентного спуска. Начальные веса сгенерированны рандомно. 

$$\boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \eta\nabla_{\boldsymbol{w}} L$$

Будьте внимательны:

* На вход алгоритму приходит $\boldsymbol{y}$ - **строка**, как и в любой sklearn алгоритм.
* Не устанавливайте количество итераций больше 1000, так как алгоритм будет долго работать.
* Как и в линейной регрессии не забудьте добавить единичный столбец справа.

Можете использовать и обычный спуск, главное чтоб по времени зашло.

### Sample 1
#### Input:
```python
X_train = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_train = np.array([1, 1, 0, 0])

model = SGDLogReg().fit(X_train, y_train)
y_pred = model.predict(np.array([[0.5, 0.5], [ -0.5,  -0.5]]))
y_prob = model.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]]))
```
#### Output:
```python
y_pred = np.array([1., 0.])
y_prob = np.array([[0.1, 0.9],  # это не точный ответ, но очень приблизительный, отличие на 0.1 - это нормально
                   [0.9, 0.1]]) # для каждого объекта возвращаем его вероятность нуля и единицы
```

In [ ]:
import numpy as np

class SGDLogReg():
    def __init__(self, max_iter=1000, eta=0.01, lamb = 1):
        self.eta = eta
        self.max_iter = max_iter
        self.lamb = lamb
        self.w_ = None
        self.coef_ = None
        
    def _stochastic_step(self, w: np.array, X: np.array, y: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        #модифицируйте функцию из предыдущей задачи
        pass
        
    def fit(self, X: np.array, y: np.array) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        n, m = X.shape
        self.w_ = np.zeros(shape=(m, 1))
        
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        self.coef_ = self.w_.reshape(-1)
        return self
    
    def predict(self, X: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def predict_proba(self, X: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import mean_absolute_error
######################################################
X_clf = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_clf = np.array([1, 1, 0, 0])
model = SGDLogReg().fit(X_clf, y_clf)

assert model.lamb == 1

assert_equal(model.predict(np.array([[-0.5, -0.5]])), np.array([0.]))
assert_equal(model.predict(np.array([[ 0.5,  0.5]])), np.array([1.]))
######################################################
np.random.seed(1337)
n = 200
a = np.random.normal(loc=0, scale=1, size=(n, 2)) #первый класс
b = np.random.normal(loc=4, scale=2, size=(n, 2)) #второй класс
X = np.vstack([a, b]) #двумерный количественный признак
y = np.hstack([np.zeros(n), np.ones(n)]) #бинарный признак

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=1645)

model = SGDLogReg(lamb=0.01).fit(X_train, y_train)
model_real = LogisticRegression(C=0.01, solver='sag').fit(X_train, y_train)

assert model.lamb == 0.01

assert mean_absolute_error(model.predict(X_test), model_real.predict(X_test)) < 0.1
assert mean_absolute_error(model.predict_proba(X_test), model_real.predict_proba(X_test)) < 0.2
######################################################


# Momentum

Этот метод позволяет направить sgd в нужной размерности и уменьшить осцилляцию. 

В общем случае он будет выглядеть следующим образом: 

$$ \boldsymbol{g}^{(t)} = \gamma \boldsymbol{g}^{(t - 1)} + \eta \nabla_{\boldsymbol{w}}L_{k}$$
$$ \boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \boldsymbol{g}^{(t)}$$

где

 - $\eta$ — learning rate
 - $\boldsymbol{w}$ — вектор параметров
 - $\boldsymbol{g}$ — вектор градиентов 
 - $L$ — оптимизируемый функционал
 - $\gamma$ — momentum term (обычно выбирается 0.9)


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDMomentum(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000):
        self.gradient = gradient #функция находящая градиент за вас
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        v = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            cur_grad = self.gradient(self.w, X[index, :], np.array(y[index]), self.l)
            # пересчитайте веса в стохаистическом градиентном спуске
            '''
            .∧＿∧ 
            ( ･ω･｡)つ━☆・*。 
            ⊂  ノ    ・゜+. 
            しーＪ   °。+ *´¨) 
                    .· ´¸.·*´¨) 
                    (¸.·´ (¸.·'* ☆  <YOUR CODE>
            '''
            self.w = 
        return self

In [ ]:
######################################################
X = np.array([[ 2.67973871e-01, -9.43407158e-01,  4.59566449e-01,
                1.63136986e-01, -5.44506820e-01]])
y = np.array([-1])

r0 = SGDMomentum(5, lambda a, b, c, d: a, max_iter=10, l=1, lr=1)
r0.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r0.fit(X, y)

assert np.allclose(r0.w.reshape(6), np.array([ 0.01753165,  0.0350633,   0.05259494,  0.0350633,   0.01753165, -0.01753165]))
######################################################
